In [ ]:
#! pip install transformers torch torchvision torchaudio

In [ ]:
import os
os.environ['HF_TOKEN'] = getToken()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from collections import Counter
import transformers
import pandas as pd
import numpy as np
import csv
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm
import gc
from collections import Counter
import re
import argparse


In [ ]:
import torch
from transformers import AutoTokenizer
from torch import nn
import joblib
from tqdm import tqdm
from sklearn.neighbors import NearestCentroid
import numpy as np
import pandas as pd
from transformers import AutoModel
import os
import threading
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

class WhosAIClassifier:
    def __init__(self, embedder, tokenizer, device):
        self.embedder = embedder
        self.multiclass_centroids = None
        self.binary_centroids = None
        self.tokenizer = tokenizer
        self.device = device

    def _get_embeddings(self, texts, features, batch_size=32):
        embeddings_list = []
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i + batch_size]
            batch_features = features[i:i + batch_size]
            encoding = self.tokenizer(
                batch_texts,
                truncation=True,
                max_length=512,
                padding='max_length',
                return_tensors='pt'
            )

            input_ids = encoding['input_ids'].to(self.device)
            attention_mask = encoding['attention_mask'].to(self.device)
            batch_features_tensor = torch.tensor(batch_features, dtype=torch.float32).to(self.device)

            with torch.no_grad():
                embeddings = self.embedder(input_ids, attention_mask, batch_features_tensor).detach().cpu().numpy()
            embeddings_list.append(embeddings)
        return np.concatenate(embeddings_list, axis=0)

    def fit(self, texts, labels, features,batch_size=32):
        binary_labels = [1 if label == 'human' else 0 for label in labels]
        embeddings = self._get_embeddings(texts, features,batch_size)
        self.multiclass_centroids = NearestCentroid().fit(embeddings, labels)
        self.binary_centroids = NearestCentroid().fit(embeddings, binary_labels)
        return self

    def predict(self, texts, features, batch_size=16):
        predictions_multi = []
        predictions_binary = []

        chunk_size = 1000
        for i in range(0, len(texts), chunk_size):
            chunk_end = min(i + chunk_size, len(texts))
            chunk_texts = texts[i:chunk_end]
            chunk_features = features[i:chunk_end]

            embeddings = self._get_embeddings(chunk_texts, chunk_features, batch_size)
            chunk_pred_multi = self.multiclass_centroids.predict(embeddings)
            chunk_pred_binary = self.binary_centroids.predict(embeddings)

            predictions_multi.extend(chunk_pred_multi)
            predictions_binary.extend(chunk_pred_binary)

            del embeddings
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        return np.array(predictions_multi), np.array(predictions_binary)


    def save(self, path="classifier.joblib"):
        joblib.dump(self, path)

    @staticmethod
    def load(path="classifier.joblib"):
        return joblib.load(path)

class AuthorAttributionModel(nn.Module):
    def __init__(self, model_name="bert-base-uncased"):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)

        self.text_projection = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

        self.uid_weight_net = nn.Sequential(
            nn.Linear(5, 16),
            nn.ReLU(),
            nn.Linear(16, 128),
            nn.Sigmoid()
        )

    def forward(self, input_ids, attention_mask, uid_features):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        text_embeddings = outputs.last_hidden_state[:, 0, :]

        text_embeddings = self.text_projection(text_embeddings)
        text_embeddings = nn.functional.normalize(text_embeddings, p=2, dim=1)

        uid_weights = self.uid_weight_net(uid_features)

        weighted_embeddings = text_embeddings * uid_weights
        final_embeddings = nn.functional.normalize(weighted_embeddings, p=2, dim=1)

        return final_embeddings



class InferenceHelper:
    def __init__(self, model_path="author_attribution_model1.pt", classifier_path="whosai_classifier_1.joblib"):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.model = AuthorAttributionModel()
        self.model.load_state_dict(torch.load(model_path, map_location=self.device, weights_only=True))
        self.model.to(self.device)
        self.model.eval()
        self.classifier = joblib.load(classifier_path)
        self.classifier.embedder = self.model
        self.classifier.device = self.device
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

    def get_embedding(self, text,features):
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=512,
            padding='max_length',
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        features = features.to(self.device)
        with torch.no_grad():
            embedding = self.model(input_ids, attention_mask,features)

        return embedding.cpu().numpy()

    def predict_single(self, text,features):
        embedding = self.get_embedding(text,features)
        multiclass_pred = self.classifier.multiclass_centroids.predict(embedding)
        binary_pred = self.classifier.binary_centroids.predict(embedding)
        binary_label = "human" if binary_pred[0] == 1 else "non-human"

        return {
            "multiclass_prediction": multiclass_pred[0],
            "binary_prediction": binary_label,
            "embedding": embedding[0]
        }
    def predict_without_features(self,text):
        features = gptwho_get_features(text)
        features = torch.tensor(features,dtype=torch.float32)
        return self.predict_single(text,features)



def Inference(model_path,classifier_path):
    try:
        inference = InferenceHelper(model_path,classifier_path)
        print("Model and classifier loaded successfully!")
    except Exception as e:
        print(f"Error loading model or classifier: {e}")
        return

    while True:
        print("\nEnter a text to classify (or 'quit' to exit):")
        text = input()

        if text.lower() == 'quit':
            break
        features = gptwho_get_features(text)
        features = torch.tensor(features,dtype=torch.float32)
        try:
            predictions = inference.predict_single(text,features)
            print("\nPredictions:")
            print(f"Detailed class: {predictions['multiclass_prediction']}")
            print(f"Binary class: {predictions['binary_prediction']}")
        except Exception as e:
            print(f"Error making prediction: {e}")


def evaluate_whosai(inference_helper, test_csv_path, batch_size=16):

    print("Loading test data...")
    data = pd.read_csv('uid_features_test.csv')
    texts = data['text'].tolist()
    features = data[['uid_var', 'uid_diff', 'uid_diff2', 'mean', 'sum']].values
    true_labels = data['label'].tolist()
    print("\nGenerating predictions...")
    multiclass_pred, binary_pred = inference_helper.classifier.predict(texts, features, batch_size=batch_size)
    binary_pred_labels = ['human' if pred == 1 else 'non-human' for pred in binary_pred]
    binary_true_labels = ['human' if label == 'human' else 'non-human' for label in true_labels]

    print("\nMulticlass Classification Report:")
    print(classification_report(true_labels, multiclass_pred))

    print("\nBinary Classification Report (Human vs Non-Human):")
    print(classification_report(binary_true_labels, binary_pred_labels))

    results_df = pd.DataFrame({
        'Text': texts,
        'True_Label': true_labels,
        'Predicted_Multiclass': multiclass_pred,
        'Predicted_Binary': binary_pred_labels
    })
    results_df.to_csv('evaluation_results_2.csv', index=False)
    plot_confusion_matrices(true_labels, multiclass_pred, binary_true_labels, binary_pred_labels)
    print("\nDetailed results saved to 'evaluation_results.csv'")

def Test():
    try:
        # Set memory management parameters
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.backends.cudnn.benchmark = False

        model_path, classifier_path = "/content/drive/MyDrive/Whos AI/Final Results/author_attribution_model.pt", "/content/drive/MyDrive/Whos AI/Final Results/whosai_classifier.joblib"
        Inference(model_path, classifier_path)
        print("Model and classifier loaded successfully!")

    except Exception as e:
        print(f"Error during evaluation: {e}")

if __name__ == "__main__":
    Test()


Error loading model or classifier: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.
Model and classifier loaded successfully!


In [ ]:
!ls '/content/drive/MyDrive/Whos AI/'

 Dataset  'Final Results'  'Other Results'   Papers


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr
import pandas as pd
import random

df = pd.read_csv("test.csv")
df.head()

ParserError: Error tokenizing data. C error: EOF inside string starting at row 2465

In [ ]:
global feature_list = []

def get_random_row(category):
    global feature_list = []
    category_data = df[df['label'] == category]
    if not category_data.empty:
        random_row = random.choice(category_data['Generation'].tolist())
        return random_row
    return "No data available for this category."

def process_output(input_context):
  try:
    model_path = "/content/drive/MyDrive/Whos AI/experiment 3/author_attribution_model1.pt"
    classifier_path = "/content/drive/MyDrive/Whos AI/experiment 3/whosai_classifier_1.joblib"
    inference = InferenceHelper(model_path,classifier_path)
    predictions = inference.predict_single(input_context,features)
    result = f"Detailed class: {predictions['multiclass_prediction']}" + "\n" + f"Binary class: {predictions['binary_prediction']}"
    return result
  except Exception as e:
        print(f"Error loading model or classifier: {e}")

def create_gradio_interface():
    with gr.Blocks() as app:
        # Dropdown to select a category
        category_dropdown = gr.Dropdown(choices=df['label'].unique().tolist(), label="Select LLM")

        # Submit button
        submit_button = gr.Button("Get generation Context")

        # Textbox to display the output
        output_textbox = gr.Textbox(label="Random Output")

        # Define action on button click
        submit_button.click(
            fn=get_random_row,
            inputs=category_dropdown,  # Pass dropdown value as input
            outputs=output_textbox  # Output to the textbox
        )

        submit_button_2 = gr.Button("Process Output")

        processed_output_textbox = gr.Textbox(label="Processed Output", interactive=False)

        submit_button_2.click(
            fn=process_output,  # Call the function to process the output
            inputs=output_textbox,  # Pass the output of the first textbox as input
            outputs=processed_output_textbox  # Output to the second textbox
        )

    return app

In [ ]:
def gptwho_get_features(sentence):
    device = torch.device('cuda')
    print("CUDA status:", torch.cuda.is_available())

    all_sents = sentence

    def local_diff(x):
        d = 0
        for i in range(len(x)-1):
            d += abs(x[i+1]-x[i])
        return d/len(x)

    def local_diff2(x):
        d = 0
        for i in range(len(x)-1):
            d += (x[i+1]-x[i])**2
        return d/len(x)



    tokenizer_class, model_class = GPT2Tokenizer, GPT2LMHeadModel
    tokenizer = tokenizer_class.from_pretrained("gpt2-xl")
    model = model_class.from_pretrained("gpt2-xl").to(device)
    tokenizer.pad_token = tokenizer.eos_token

    # Getting UID_var and all surprisals for "UID spans" features
    def get_line_uid_surp(lines):
        with torch.no_grad():
            lines = tokenizer.eos_token + lines
            tok_res = tokenizer(lines, return_tensors='pt')
            input_ids = tok_res['input_ids'][0].to(device)
            attention_mask = tok_res['attention_mask'][0].to(device)
            lines_len = torch.sum(tok_res['attention_mask'], dim=1)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss, logits = outputs[:2]
        line_log_prob = torch.tensor([0.0])
        word_probs = []
        for token_ind in range(lines_len - 1):
            token_prob = F.softmax(logits[token_ind], dim=0).to('cpu')
            token_id = input_ids[token_ind + 1].to('cpu')
            target_log_prob = -torch.log(token_prob[token_id]).detach().cpu().numpy()
            line_log_prob += -torch.log(token_prob[token_id]).detach().cpu().numpy()
            word_probs.append(target_log_prob)

        # mu = average surprisal
        mu = line_log_prob/(lines_len-1)
        uid = torch.tensor([0.0])

        for i in range(len(word_probs)):
            uid += (word_probs[i] - mu)**2/(len(word_probs))
        sentence_uids = uid.detach().cpu().numpy()[0]
        sentence_surprisal = np.mean(word_probs)
        sentence_length = lines_len.detach().cpu().numpy()[0]
        torch.cuda.empty_cache()
        return(sentence_uids, sentence_surprisal, word_probs, sentence_length)

    #row=["text", "label", "uid_var", "uid_diff", "uid_diff2", "mean", "sum", "surps", "n_token"]

    uids, surps, probs, lens = get_line_uid_surp(sentence)
    uid_diff1 = local_diff(probs)
    uid_diff2 = local_diff2(probs)
    sum = np.sum(probs)
    gc.collect()
    torch.cuda.empty_cache()
    #row_=[batch, labels, uids, uid_diff1, uid_diff2, surps, sum, probs, lens]
    #print(row_)
    features = [uids,uid_diff1,uid_diff2,surps,sum]
    return features



In [ ]:
# import gradio as gr
# import pandas as pd
# import random

# # Load the dataset
# df = pd.read_csv("test.csv")  # Ensure test.csv contains 'sentence' and 'label' columns

# print(df.head())

# # Function to fetch a random sentence for a selected LLM
# def get_random_sentence(llm_name):
#     llm_data = df[df["label"] == llm_name]
#     if llm_data.empty:
#         return "No data available for the selected LLM."
#     return random.choice(llm_data["Generation"].tolist())

# # Function to process the prediction
# def process_prediction(input_context):
#   try:
#     model_path = "/content/drive/MyDrive/Whos AI/Turing Bench/author_attribution_model.pt"
#     classifier_path = "/content/drive/MyDrive/Whos AI/Turing Bench/whosai_classifier_0.joblib"
#     inference_helper = InferenceHelper(model_path, classifier_path)
#     predictions = inference_helper.predict_single(input_context)
#     return predictions["multiclass_prediction"]
#   except Exception as e:
#     print(f"Error during prediction: {e}")
#     return f"Error during prediction: {e}"

# # Create the Gradio interface
# def create_gradio_interface():
#     llm_names = df["label"].unique()

#     # Text boxes
#     input_textbox = gr.Textbox(label="Context", interactive=False, lines=5)
#     prediction_box = gr.Textbox(label="Your Prediction", interactive=True)
#     output_box = gr.Textbox(label="Output")

#     # Interface Layout
#     with gr.Blocks() as app:
#         with gr.Row():
#             gr.Markdown("## LLM Selector and Predictor")

#         with gr.Row():
#             for llm_name in llm_names:
#                 # Correctly pass the current value of llm_name
#                 gr.Button(llm_name).click(
#                     fn=lambda llm=llm_name: get_random_sentence(llm),
#                     inputs=None,
#                     outputs=input_textbox
#                 )

#         with gr.Row():
#             gr.Markdown("### Enter Your Prediction")
#             prediction_box.render()

#         with gr.Row():
#             submit_button = gr.Button("Submit Prediction")
#             submit_button.click(
#                 fn=process_prediction, inputs=input_textbox, outputs=output_box
#             )

#         with gr.Row():
#             gr.Markdown("### Model Output")
#             output_box.render()

#     return app

# if __name__ == "__main__":
#     app = create_gradio_interface()
#     app.launch(debug=True)


In [ ]:
!ls /content/drive/MyDrive

ls: cannot access '/content/drive/MyDrive': No such file or directory


In [ ]:
!pip install dill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer
from torch import nn
from io import BytesIO
import joblib
from tqdm import tqdm
from sklearn.neighbors import NearestCentroid
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

print(torch.__version__)
print(torch.cuda.is_available())
# import os
# import threading

# os.environ['OPENBLAS_NUM_THREADS'] = '64'
# os.environ['OMP_NUM_THREADS'] = '64'
# threading.stack_size(2**26)

class WhosAIClassifier:
    def __init__(self, embedder, tokenizer, device):
        self.embedder = embedder
        self.multiclass_centroids = None
        self.binary_centroids = None
        self.tokenizer = tokenizer
        self.device = device

    def _get_embeddings(self, texts, batch_size=32):
        embeddings_list = []
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i + batch_size]
            encoding = self.tokenizer(batch_texts,
                                    truncation=True,
                                    max_length=512,
                                    padding='max_length',
                                    return_tensors='pt')
            input_ids = encoding['input_ids'].to(self.device)
            attention_mask = encoding['attention_mask'].to(self.device)

            with torch.no_grad():
                embeddings = self.embedder(input_ids, attention_mask).detach().cpu().numpy()
            embeddings_list.append(embeddings)

        return np.concatenate(embeddings_list, axis=0)

    def fit(self, texts, labels, batch_size=32):
        binary_labels = [1 if label == 'human' else 0 for label in labels]
        embeddings = self._get_embeddings(texts, batch_size)
        self.multiclass_centroids = NearestCentroid().fit(embeddings, labels)
        self.binary_centroids = NearestCentroid().fit(embeddings, binary_labels)
        return self

    def predict(self, texts, batch_size=32):
        embeddings = self._get_embeddings(texts, batch_size)
        multiclass_predictions = self.multiclass_centroids.predict(embeddings)
        binary_predictions = self.binary_centroids.predict(embeddings)
        return multiclass_predictions, binary_predictions

    def save(self, path="classifier.joblib"):
        joblib.dump(self, path)

    @staticmethod
    def load(path="classifier.joblib"):
        return joblib.load(path)

class AuthorAttributionModel(nn.Module):
    def __init__(self, model_name="bert-base-uncased"):
        super().__init__()
        from transformers import AutoModel
        self.bert = AutoModel.from_pretrained(model_name)
        self.projection = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state[:, 0, :]
        embeddings = self.projection(embeddings)
        embeddings = nn.functional.normalize(embeddings, p=2, dim=1)
        return embeddings

class InferenceHelper:
    def __init__(self, model_path="author_attribution_model.pt", classifier_path="whosai_classifier.joblib"):
        # Determine the best available device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Create a flexible map_location function
        def flexible_map_location(storage, location):
            # This will map all previous device storages to the current available device
            return storage.cuda(self.device.index) if self.device.type == 'cuda' else storage.cpu()

        # Load the model with flexible device mapping
        self.model = AuthorAttributionModel()
        self.model.load_state_dict(torch.load(model_path, map_location=flexible_map_location, weights_only=True))
        self.model.to(self.device)
        self.model.eval()

        self.classifier = joblib.load(classifier_path)
        self.classifier.embedder = self.model
        self.classifier.device = self.device
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


    def get_embedding(self, text):
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=512,
            padding='max_length',
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)

        with torch.no_grad():
            embedding = self.model(input_ids, attention_mask)

        return embedding.cpu().numpy()

    def predict_single(self, text):
        embedding = self.get_embedding(text)
        multiclass_pred = self.classifier.multiclass_centroids.predict(embedding)
        binary_pred = self.classifier.binary_centroids.predict(embedding)
        binary_label = "human" if binary_pred[0] == 1 else "non-human"

        return {
            "multiclass_prediction": multiclass_pred[0],
            "binary_prediction": binary_label,
            "embedding": embedding[0]
        }


def Inference(model_path, classifier_path):
    try:
        inference = InferenceHelper(model_path, classifier_path)
        print("Model and classifier loaded successfully!")
    except Exception as e:
        print(f"Error loading model or classifier: {e}")
        return

    while True:
        print("\nEnter a text to classify (or 'quit' to exit):")
        text = input()

        if text.lower() == 'quit':
            break

        try:
            predictions = inference.predict_single(text)
            print("\nPredictions:")
            print(f"Detailed class: {predictions['multiclass_prediction']}")
            print(f"Binary class: {predictions['binary_prediction']}")
        except Exception as e:
            print(f"Error making prediction: {e}")

def evaluate_whosai(model_path, classifier_path, test_csv_path, batch_size=32):
    inference_helper = InferenceHelper(model_path, classifier_path)
    print("Model and classifier loaded successfully!")
    print("Loading test data...")
    test_data = pd.read_csv(test_csv_path)
    texts = test_data['Generation'].tolist()
    true_labels = test_data['label'].tolist()
    print("\nGenerating predictions...")
    multiclass_pred, binary_pred = inference_helper.classifier.predict(texts, batch_size=batch_size)
    binary_pred_labels = ['human' if pred == 1 else 'non-human' for pred in binary_pred]
    binary_true_labels = ['human' if label == 'human' else 'non-human' for label in true_labels]

    print("\nMulticlass Classification Report:")
    print(classification_report(true_labels, multiclass_pred))

    print("\nBinary Classification Report (Human vs Non-Human):")
    print(classification_report(binary_true_labels, binary_pred_labels))

    results_df = pd.DataFrame({
        'Text': texts,
        'True_Label': true_labels,
        'Predicted_Multiclass': multiclass_pred,
        'Predicted_Binary': binary_pred_labels
    })
    results_df.to_csv('evaluation_results.csv', index=False)
    print("\nDetailed results saved to 'evaluation_results.csv'")

def Test():
    try:
        # model_path, classifier_path, test_path = "author_attribution_model.pt", "whosai_classifier.joblib", "test.csv"
        # evaluate_whosai(model_path, classifier_path, test_path, batch_size=32)
        model_path, classifier_path = "/content/drive/MyDrive/Whos AI/Turing Bench/author_attribution_model.pt", "/content/drive/MyDrive/Whos AI/Turing Bench/whosai_classifier_0.joblib"
        Inference(model_path, classifier_path)

    except Exception as e:
        print(f"Error during evaluation: {e}")

if __name__ == "__main__":
    Test()


2.5.1+cu121
True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model and classifier loaded successfully!

Enter a text to classify (or 'quit' to exit):
quit
